# Graph generator

- import the inferenced point clouds
- import the doors 
- export graphs

## LIBRARIES

In [1]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy

import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

#import utils
import context 
import utils as utl
import utils.t1_utils as t1

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [5]:
#Currect directory
current_directory=Path(os.getcwd())
print(current_directory)

name = 'school'

path = Path(os.getcwd()).parents[0] / 'data'

pcd_input_path = path / name
print('pcd_input_path:', pcd_input_path)

if not pcd_input_path.exists():
    print(f"Directory not found: {pcd_input_path}")
else:
    print(f"Directory exists: {pcd_input_path}")

class_file = Path(os.getcwd()) / '_classes_expanded.json'
print('class_file:', class_file)

# Check if the class file exists
if not class_file.exists():
    print(f"Class file not found: {class_file}")
else:
    print(f"Class file exists: {class_file}")


c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Scan-to-BIM-CVPR-2024\code\_Scan-to-BIM-CVPR-2024-OR\scripts
pcd_input_path: c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Scan-to-BIM-CVPR-2024\code\_Scan-to-BIM-CVPR-2024-OR\data\school
Directory not found: c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Scan-to-BIM-CVPR-2024\code\_Scan-to-BIM-CVPR-2024-OR\data\school
class_file: c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Scan-to-BIM-CVPR-2024\code\_Scan-to-BIM-CVPR-2024-OR\scripts\_classes_expanded.json
Class file not found: c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Scan-to-BIM-CVPR-2024\code\_Scan-to-BIM-CVPR-2024-OR\scripts\_classes_expanded.json


OUTPUT

In [6]:
# name=name.split('_')[0]
output_folder = Path(os.getcwd()).parents[0] / 'outputs'
json_output_path=os.path.join(output_folder, f'{name}_elemnts.json') 
geometry_output_path= os.path.join(output_folder, f'{name}_elements.obj') # these are the bounding surfaces of the reference levels (optional)
print(pcd_input_path)

file_graph_name = 'school'
graphPath = str(output_folder / f'{name}_graph.ttl')
print('Output folder', output_folder)

c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Scan-to-BIM-CVPR-2024\code\_Scan-to-BIM-CVPR-2024-OR\data\school
Output folder c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Scan-to-BIM-CVPR-2024\code\_Scan-to-BIM-CVPR-2024-OR\outputs


### IMPORT PCD

GET FILES

In [7]:
def get_list_of_files(folder: Path | str , ext: str = None) -> list:
    """
    Get a list of all filepaths in the folder and subfolders that match the given file extension.

    Args:
        folder: The path to the folder as a string or Path object
        ext: Optional. The file extension to filter by, e.g., ".txt". If None, all files are returned.

    Returns:
        A list of filepaths that match the given file extension.
    """
    folder = Path(folder)  # Ensure the folder is a Path object
    allFiles = []
    # Iterate over all the entries in the directory
    for entry in folder.iterdir():
        # Create full path
        fullPath = entry
        # If entry is a directory then get the list of files in this directory 
        if fullPath.is_dir():
            allFiles += get_list_of_files(fullPath, ext=ext)
        else:
            # Check if file matches the extension
            if ext is None or fullPath.suffix.lower() == ext.lower():
                allFiles.append(fullPath.as_posix())
    return allFiles

In [15]:
# List all .laz files in the directory
def get_list_of_files(folder, extension):
    return [str(file) for file in Path(folder).glob(f'*{extension}')]

input_ = path / 'school'
print(input_)
files = get_list_of_files(input_, '.laz')

# Check if any files were found
if not files:
    print("No .laz files found.")
else:
    print('Input files:', files)

c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Scan-to-BIM-CVPR-2024\code\_Scan-to-BIM-CVPR-2024-OR\data\school
No .laz files found.


In [11]:
files = os.path.join(pcd_input_path , 'school.laz')
#files='32_05_01_test.las'
# utl.get_list_of_files(output_folder, '.laz')

# for each file in files
print('Input file: ', files)

# # for each file in files
# print(len(file))
# print(pcd_input_path)

Input file:  c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Scan-to-BIM-CVPR-2024\code\_Scan-to-BIM-CVPR-2024-OR\data\school\school.laz


In [12]:
laz=laspy.read(files)

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\oscar\\OneDrive - Fondazione Bruno Kessler\\Scan-to-BIM-CVPR-2024\\code\\_Scan-to-BIM-CVPR-2024-OR\\data\\school\\school.laz'

In [39]:
import json

class_file=r'C:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Scan-to-BIM-CVPR-2024\data\challenge_laz\_classes.json'

# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': -1, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 0, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 1, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 2, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 3, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 4, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 5, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


In [28]:
# from geomapi.nodes import PointCloudNode

# pcdNodes=[]

# # for f in files[0:1]:
# laz  = laspy.read(files)
# pcd=gmu.las_to_pcd(laz)
# pcdNodes.append(PointCloudNode(resource=pcd,
#                             name=ut.get_filename(files)))
# # del laz
# # del pcd
# print(f'created {pcdNodes[-1].name}: {pcdNodes[-1].resource}')

created 32_05_01_test: PointCloud with 3928910 points.


In [40]:
from geomapi.nodes import PointCloudNode

In [41]:
pcdNodes=[]
# import pcd
print(f'processing {files}')
# laz  = laspy.read(f) #enable this
print(f'loaded {len(laz.xyz)} points')

#seperate classes
for c in class_dict['classes']:
    idx=np.where((laz['classes']==c['id']))
    objects_ids=laz['objects'][idx]        
    class_points=laz.xyz[idx]
    
    #separate objects
    for u in np.unique(objects_ids):
        idx=np.where(objects_ids==u)
        object_pcd=o3d.geometry.PointCloud()
        object_pcd.points=o3d.utility.Vector3dVector(class_points[idx])
        pcdNode=PointCloudNode(resource=object_pcd,
                                name=ut.get_filename(files)+'_'+c['name']+'_'+str(u),
                                class_name=c['name'],
                                class_id=c['id'],
                                object_id=u,
                                color=ut.random_color(),
                                derived_from=ut.get_filename(files))
        pcdNode.get_oriented_bounding_box()
        pcdNodes.append(pcdNode)
# del laz
# del class_points
print(f'created {len(pcdNodes)} from {ut.get_filename(files)}')


processing c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Scan-to-BIM-CVPR-2024\data\challenge_laz\32_05_01_test.laz
loaded 3459902 points
created 31 from 32_05_01_test


In [34]:
# laz= laspy.read(files)
# pcd=gmu.las_to_pcd(laz)
# pcdNodes.append(PointCloudNode(resource=pcd))
# pcd=gmu.las_to_pcd(laz)
# o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


### import entire folder of PCD's

## OUTPUTS

### export graphs

In [42]:
name= '32_05_01_test'

print('Output path: ', output_folder)
output = os.path.join(output_folder , f'{name}_graph.ttl')
print(output)

tl.nodes_to_graph(pcdNodes,
                graphPath=output,
                save=True)

Output path:  c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Scan-to-BIM-CVPR-2024\data\32_02_05_tests
c:\Users\oscar\OneDrive - Fondazione Bruno Kessler\Scan-to-BIM-CVPR-2024\data\32_02_05_tests\32_05_01_test_graph.ttl


<Graph identifier=N22147ced0f5449f88f223805264fbb87 (<class 'rdflib.graph.Graph'>)>

In [28]:
import copy
import scipy.spatial.transform import Rotation as R
import laspy 

laz=laspy.read(pcd_input_path)
z_values=laz[:,2]
z_values=laz.z

highest_z=np.max(z_values)
lowest_z=np.min(z_values)

delta_levels= (highest_z-lowest_z)

print(f'Highest z value: {highest_z}')
print(f'Lowest z value: {lowest_z}')
print(f'Delta z: {delta_levels}')

SyntaxError: invalid syntax (779750090.py, line 2)

In [29]:
for n in columnNodes:
   
    #compute the oriented bounding box
    n.box=n.get_oriented_bounding_box()
    n.box.color=[1,0,0]
 
    #translate the resource a bit higher
    pcd= copy.deepcopy(n.resource)
    pcd.translate([0,0,5])
   
    #combine these point clouds
    pcd+=n.resource
    n.box2=pcd.get_oriented_bounding_box()
    n.box2.color=[0,1,0]
   
    #retrieve the rotation of the box
    rotation_matrix = np.array(n.box2.R, copy=True)  # Create a writable copy
 
    #report the rotation matrix as euler angles using scipy.spatial.transform.Rotation
    r = R.from_matrix(rotation_matrix)
    euler_angles = r.as_euler('zyx', degrees=True)
    print(euler_angles)
   
    #create rotation matrix from Z=-euler_angles , Y=0, X=0
    r = R.from_euler('zyx', [-euler_angles[1],0,0], degrees=True)
    rotation_matrix = r.as_matrix()
   
    #rotating the box back to the original position
    n.box3=copy.deepcopy(n.box2)
    n.box3.rotate(r.as_matrix(  ))
    n.box3.color=[0,0,1]

NameError: name 'columnNodes' is not defined

In [65]:

 
# for n in columnNodes:
   
#     #compute the oriented bounding box
#     n.box=n.get_oriented_bounding_box()
#     n.box.color=[1,0,0]
 
#     #translate the resource a bit higher
#     pcd= copy.deepcopy(n.resource)
#     pcd.translate([0,0,5])
   
#     #combine these point clouds
#     pcd+=n.resource
#     n.box2=pcd.get_oriented_bounding_box()
#     n.box2.color=[0,1,0]
   
#     #retrieve the rotation of the box
#     rotation_matrix = np.array(n.box2.R, copy=True)  # Create a writable copy
 
#     #report the rotation matrix as euler angles using scipy.spatial.transform.Rotation
#     r = R.from_matrix(rotation_matrix)
#     euler_angles = r.as_euler('zyx', degrees=True)
#     print(euler_angles)
   
#     #create rotation matrix from Z=-euler_angles , Y=0, X=0
#     r = R.from_euler('zyx', [-euler_angles[1],0,0], degrees=True)
#     rotation_matrix = r.as_matrix()
   
#     #rotating the box back to the original position
#     n.box3=copy.deepcopy(n.box2)
#     n.box3.rotate(r.as_matrix(  ))
#     n.box3.color=[0,0,1]




# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(n.color) for n in columnNodes if n.resource is not None])
# joined_boxes=gmu.join_geometries([n.box for n in columnNodes if n.resource is not None])
 
# o3d.visualization.draw_geometries([joined_pcd]+[n.box for n in columnNodes]+[n.box2 for n in columnNodes]+[n.box3 for n in columnNodes])
# # o3d.visualization.draw_geometries([joined_pcd]+[columnNodes[0].box,columnNodes[0].box2,columnNodes[0].box3])
 

#  for f in files[0:1]:
#     pcdNodes=[]
#     # import pcd
#     print(f'processing {f}')
#     # laz  = laspy.read(f) #enable this
#     print(f'loaded {len(laz.xyz)} points')
 
#     #seperate classes
#     for c in class_dict['classes']:
#         idx=np.where((laz['classes']==c['id']))
#         objects_ids=laz['objects'][idx]        
#         class_points=laz.xyz[idx]
       
#         #separate objects
#         for u in np.unique(objects_ids):
#             idx=np.where(objects_ids==u)
#             object_pcd=o3d.geometry.PointCloud()
#             object_pcd.points=o3d.utility.Vector3dVector(class_points[idx])
#             pcdNode=PointCloudNode(resource=object_pcd,
#                                     name=ut.get_filename(f)+'_'+c['name']+'_'+str(u),
#                                     class_name=c['name'],
#                                     class_id=c['id'],
#                                     object_id=u,
#                                     color=ut.random_color(),
#                                     derived_from=ut.get_filename(f))
#             pcdNode.get_oriented_bounding_box()
#             pcdNodes.append(pcdNode)
#     # del laz
#     # del class_points
#     print(f'created {len(pcdNodes)} from {ut.get_filename(f)}')
 

#  # Read the JSON file
# with open(class_file, 'r') as file:
#     json_data = json.load(file)
 
# # Create a dictionary
# class_dict = {
#     'classes': json_data['classes'],
#     'default': json_data['default'],
#     'type': json_data['type'],
#     'format': json_data['format'],
#     'created_with': json_data['created_with']
# }
# print(class_dict)